# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies 
import pandas as pd
import numpy as np

# Files to Load 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"


In [2]:
# Read School and Student Data File and store into Pandas Data Frames
school_data_pd = pd.read_csv(school_data_to_load)
student_data_pd = pd.read_csv(student_data_to_load)
# Combine the data into a single dataset
schdc = pd.merge(student_data_pd, school_data_pd, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
schoolCount=len(schdc["school_name"].unique())
studentCount=schdc["student_name"].count()
budgetTotal=schdc["budget"].unique().sum()

In [4]:
avgMath=schdc["math_score"].mean()
avgRead=schdc["reading_score"].mean()

In [5]:
pctPassMath=(schdc["math_score"].loc[(schdc["math_score"]>=70)].count() *100)/ studentCount
pctPassRead=(schdc["reading_score"].loc[(schdc["reading_score"]>=70)].count() *100)/ studentCount
overallPassRate = (avgMath+avgRead)/2

In [6]:
#create Data Frame with all data
districtSummary = pd.DataFrame({"Total Schools":[schoolCount],
                                "Total Students":[studentCount],
                                "Total Budget":[budgetTotal],
                                "Average Math Score":[avgMath],
                                "Average Reading Score":[avgRead],
                                "% Passing Math":[pctPassMath],
                                "% Passing Reading":[pctPassRead],
                                "% Overall Passing Rate":[overallPassRate] })

In [7]:
#format data frame columns
districtSummary["Total Students"]=districtSummary["Total Students"].map("{0:,.0f}".format)
districtSummary["Total Budget"]= districtSummary["Total Budget"].map("${0:,.2f}".format)
districtSummary["Average Math Score"]=districtSummary["Average Math Score"].map("{0:,.2f}".format)
districtSummary["Average Reading Score"]=districtSummary["Average Reading Score"].map("{0:.2f}".format)
districtSummary["% Passing Math"]=districtSummary["% Passing Math"].map("{0:.2f}".format)
districtSummary["% Passing Reading"]=districtSummary["% Passing Reading"].map("{0:.2f}".format)
districtSummary["% Overall Passing Rate"]=districtSummary["% Overall Passing Rate"].map("{0:.2f}".format)                   
districtSummary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,80.43


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [8]:
school_data_pd = pd.read_csv(school_data_to_load)
#get original school data, #make index school name and add column budget per student
sch_df= school_data_pd
sch_dfi=sch_df.set_index("school_name")
sch_dfi.drop(["School ID"], axis=1, inplace=True)
sch_dfi["psb"]=sch_dfi["budget"] / sch_dfi["size"]
sch_dfi.rename(columns={"type":"School Type",
                        "size":"Total Students",
                        "budget":"Total School Budget",
                        "psb":"Per Student Budget"}, inplace=True)
#sch_dfi.head()


In [9]:
student_data_pd = pd.read_csv(student_data_to_load)
#get original student data, group by school name , calculate aggregates 
stu_df= student_data_pd
#count pass math and reading 
stu_df["passM"]=stu_df["math_score"].apply(lambda x: 0 if x < 70 else 1)
stu_df["passR"]=stu_df["reading_score"].apply(lambda x: 0 if x < 70 else 1)
stu_grp = stu_df.groupby("school_name")

stu_grp_df=pd.DataFrame({"Average Math Score":stu_grp["math_score"].mean(),
                         "Average Reading Score":stu_grp["reading_score"].mean(),
                         "% Passing Math":(stu_grp["passM"].sum() / stu_grp["school_name"].size())*100,
                         "% Passing Reading":(stu_grp["passR"].sum() / stu_grp["school_name"].size())*100,
                        })


In [10]:
allData=pd.merge(sch_dfi, stu_grp_df,  on="school_name")
allData["% Overall Passing Rate"]= (allData["% Passing Math"]+allData["% Passing Reading"])/2
bestSchs=allData.sort_values(by="% Overall Passing Rate", ascending=False)
#Format
bestSchs["Total School Budget"] = bestSchs["Total School Budget"].map("${0:,.2f}".format)
bestSchs["Per Student Budget"] = bestSchs["Per Student Budget"].map("${0:,.2f}".format)
bestSchs["Average Math Score"] = bestSchs["Average Math Score"].map("{0:.2f}".format)
bestSchs["Average Reading Score"]=bestSchs["Average Reading Score"].map("{0:.2f}".format)
bestSchs["% Passing Math"]=bestSchs["% Passing Math"].map("{0:.2f}".format)
bestSchs["% Passing Reading"]= bestSchs["% Passing Reading"].map("{0:.2f}".format)
bestSchs["% Overall Passing Rate"]=bestSchs["% Overall Passing Rate"].map("{0:.2f}".format)

bestSchs.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,95.59
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,95.29
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,95.27
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,95.27
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,95.20


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [11]:
worstSchs=allData.sort_values(by="% Overall Passing Rate")
#Format
worstSchs["Total School Budget"] = worstSchs["Total School Budget"].map("${0:,.2f}".format)
worstSchs["Per Student Budget"] = worstSchs["Per Student Budget"].map("${0:,.2f}".format)
worstSchs["Average Math Score"] = worstSchs["Average Math Score"].map("{0:.2f}".format)
worstSchs["Average Reading Score"]=worstSchs["Average Reading Score"].map("{0:.2f}".format)
worstSchs["% Passing Math"]=worstSchs["% Passing Math"].map("{0:.2f}".format)
worstSchs["% Passing Reading"]= worstSchs["% Passing Reading"].map("{0:.2f}".format)
worstSchs["% Overall Passing Rate"]=worstSchs["% Overall Passing Rate"].map("{0:.2f}".format)
worstSchs.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,73.29
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,73.36
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,73.64
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,73.80


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [12]:
#get original student data
student_data_pd = pd.read_csv(student_data_to_load)
#clean
student_data_forGradesM=student_data_pd.drop(['student_name', 'Student ID','gender','reading_score'], axis=1)
student_data_forGradesM=student_data_forGradesM.set_index("school_name")
#group per 
dg9=student_data_forGradesM.loc[student_data_forGradesM["grade"]=="9th"].groupby("school_name")
dfg=pd.DataFrame({"9th":dg9["math_score"].mean()})
dg10=student_data_forGradesM.loc[student_data_forGradesM["grade"]=="10th"].groupby("school_name")
dfg["10th"]=pd.DataFrame({"10th":dg10["math_score"].mean()})
dg11=student_data_forGradesM.loc[student_data_forGradesM["grade"]=="11th"].groupby("school_name")
dfg["11th"]=pd.DataFrame({"11th":dg11["math_score"].mean()})
dg12=student_data_forGradesM.loc[student_data_forGradesM["grade"]=="12th"].groupby("school_name")
dfg["12th"]=pd.DataFrame({"12th":dg12["math_score"].mean()})
#format
dfg["9th"]=dfg["9th"].map("{0:.2f}".format)
dfg["10th"]=dfg["10th"].map("{0:.2f}".format)
dfg["11th"]=dfg["11th"].map("{0:.2f}".format)
dfg["12th"]=dfg["12th"].map("{0:.2f}".format)
#display
dfg.head()


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [13]:
#get original student data
student_data_pd = pd.read_csv(student_data_to_load)
#clean
student_data_forGradesM=student_data_pd.drop(['student_name', 'Student ID','gender','math_score'], axis=1)
student_data_forGradesM=student_data_forGradesM.set_index("school_name")
#group per 
dg9=student_data_forGradesM.loc[student_data_forGradesM["grade"]=="9th"].groupby("school_name")
dfg=pd.DataFrame({"9th":dg9["reading_score"].mean()})
dg10=student_data_forGradesM.loc[student_data_forGradesM["grade"]=="10th"].groupby("school_name")
dfg["10th"]=pd.DataFrame({"10th":dg10["reading_score"].mean()})
dg11=student_data_forGradesM.loc[student_data_forGradesM["grade"]=="11th"].groupby("school_name")
dfg["11th"]=pd.DataFrame({"11th":dg11["reading_score"].mean()})
dg12=student_data_forGradesM.loc[student_data_forGradesM["grade"]=="12th"].groupby("school_name")
dfg["12th"]=pd.DataFrame({"12th":dg12["reading_score"].mean()})
#format
dfg["9th"]=dfg["9th"].map("{0:.2f}".format)
dfg["10th"]=dfg["10th"].map("{0:.2f}".format)
dfg["11th"]=dfg["11th"].map("{0:.2f}".format)
dfg["12th"]=dfg["12th"].map("{0:.2f}".format)
#display
dfg.head()


,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [14]:
spmdf=schdc.drop(['student_name', 'Student ID','gender','type', 'School ID','grade'], axis=1)
spmdf["bps"]=spmdf["budget"]//spmdf["size"]
#additional data for later
spmdf["passM"]=spmdf["math_score"].apply(lambda x: 0 if x < 70 else 1)
spmdf["passR"]=spmdf["reading_score"].apply(lambda x: 0 if x < 70 else 1)


In [17]:
# Sample bins. Feel free to create your own bins.
#minimum is $578, max is $655
spending_bins = [574, 600, 624, 649, 674]
group_names = ["$575-599", "$600-624", "$625-649", "$650-674"]
#bin according to budget per student
spmdf["Spending Ranges Per Student"]=pd.cut(spmdf["bps"], spending_bins, labels=group_names)
gspmdf= spmdf.groupby("Spending Ranges Per Student")
spendingdf=pd.DataFrame({
                         "Average Math Score":gspmdf["math_score"].mean(),
                         "Average Reading Score":gspmdf["reading_score"].mean(),
                         "% Passing Math":(gspmdf["passM"].sum() / gspmdf["Spending Ranges Per Student"].size())*100,
                         "% Passing Reading":(gspmdf["passR"].sum() / gspmdf["Spending Ranges Per Student"].size())*100,
                        })

spendingdf["% Overall Passing Rate"]=(spendingdf["% Passing Math"]+spendingdf["% Passing Reading"])/2
#format
spendingdf.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges Per Student,,,,,
$575-599,83.362283,83.912412,93.738467,96.506335,95.122401
$600-624,83.839917,84.044699,94.594595,95.945946,95.270270
$625-649,78.061635,81.434088,71.400428,83.614770,77.507599
$650-674,77.049297,81.005604,66.230813,81.109397,73.670105


## Scores by School Size

* Perform the same operations as above, based on school size.

In [18]:

# Combine the data into a single dataset
sc_sz_df= pd.merge(pd.read_csv("Resources/schools_complete.csv"), pd.read_csv("Resources/students_complete.csv"), how="left", on=["school_name", "school_name"])
#additional data for later
sc_sz_df["passM"]=sc_sz_df["math_score"].apply(lambda x: 0 if x < 70 else 1)
sc_sz_df["passR"]=sc_sz_df["reading_score"].apply(lambda x: 0 if x < 70 else 1)
#binning
size_bins = [0, 500, 1000, 2000, 3000,4000, 5000]
group_names = ["Micro(<500)", "XS(500-1000)", "S(1000-1999)", "M(2000-2999)",  "L(3000-3999)", "XL(4000-4999)"]
sc_sz_df["School Size"]=pd.cut(sc_sz_df["size"], size_bins, labels=group_names)
#grouping

g_sc_sz_df= sc_sz_df.groupby("School Size")
schoolszdf=pd.DataFrame({
                         "Average Math Score":g_sc_sz_df["math_score"].mean(),
                         "Average Reading Score":g_sc_sz_df["reading_score"].mean(),
                         "% Passing Math":(g_sc_sz_df["passM"].sum() / g_sc_sz_df["School Size"].size())*100,
                         "% Passing Reading":(g_sc_sz_df["passR"].sum() / g_sc_sz_df["School Size"].size())*100,
                        })
schoolszdf["% Overall Passing Rate"]=(schoolszdf["% Passing Math"]+schoolszdf["% Passing Reading"])/2
#print
schoolszdf.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Micro(<500),83.803279,83.814988,92.505855,96.252927,94.379391
XS(500-1000),83.839917,84.044699,94.594595,95.945946,95.270270
S(1000-1999),83.372682,83.867989,93.616522,96.773058,95.194790
M(2000-2999),78.164034,81.654758,72.336517,83.844600,78.090558
L(3000-3999),76.842711,80.744686,66.366592,80.220055,73.293323


## Scores by School Type

* Perform the same operations as above, based on school type.

In [20]:

# Combine the data into a single dataset
sc_st_df= pd.merge(pd.read_csv("Resources/schools_complete.csv"), pd.read_csv("Resources/students_complete.csv"), how="left", on=["school_name", "school_name"])
#additional data for later
sc_st_df["passM"]=sc_sz_df["math_score"].apply(lambda x: 0 if x < 70 else 1)
sc_st_df["passR"]=sc_sz_df["reading_score"].apply(lambda x: 0 if x < 70 else 1)
#group by type
g_sc_st_df= sc_st_df.groupby("type")
schooltydf=pd.DataFrame({
                         "Average Math Score":g_sc_st_df["math_score"].mean(),
                         "Average Reading Score":g_sc_st_df["reading_score"].mean(),
                         "% Passing Math":(g_sc_st_df["passM"].sum() / g_sc_st_df["type"].size())*100,
                         "% Passing Reading":(g_sc_st_df["passR"].sum() / g_sc_st_df["type"].size())*100,
                        })
schooltydf["% Overall Passing Rate"]=(schooltydf["% Passing Math"]+schooltydf["% Passing Reading"])/2
#print
schooltydf.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
